In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 16 06:42:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: keras-preprocessing, wheel, six, numpy, gast, google-pasta, typing-extensions, tensorboard, protobuf, flatbuffers, astunparse, tensorflow-estimator, keras, opt-einsum, absl-py, clang, wrapt, grpcio, h5py, termcolor
Required-by: kapre


In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


In [ ]:
!ls

drive  sample_data


In [ ]:

import os
for dirname, _, filenames in os.walk('/gdrive/MyDrive/dacon/Herbarium/'):
    for filename in filenames:
        if filename.endswith('.jpg'):
            break
        print(os.path.join(dirname, filename))
        # /gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7.zip
        # /gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/test/metadata.jso
        # /gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/metadata.jso

In [ ]:
import numpy as np 
import pandas as pd 
!ls
sample_sub = pd.read_csv('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/sample_submission.csv')
# sample_sub = pd.read_csv('../dacon/Herbarium/herbarium-2020-fgvc7/sample_submission.csv')
display(sample_sub)

drive  sample_data


,Id,Predicted
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
138287,138287,0
138288,138288,0
138289,138289,0
138290,138290,0


In [ ]:
import json, codecs
with codecs.open('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/metadata.json', 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
    
with codecs.open('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/test/metadata.json', 'r',
                 encoding='utf-8', errors='ignore') as f:
    test_meta = json.load(f)
display(train_meta.keys())
# train_df = pd.read_json('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/metadata.json',encoding='utf-8')
# print(train_df)

dict_keys(['annotations', 'categories', 'images', 'info', 'licenses', 'regions'])

In [ ]:
train_df = pd.DataFrame(train_meta['annotations'])
train_cat = pd.DataFrame(train_meta['categories'])
train_cat.columns = ['family', 'genus', 'category_id', 'category_name']
train_img = pd.DataFrame(train_meta['images'])
train_img.columns = ['file_name', 'height', 'image_id', 'license', 'width']
train_reg = pd.DataFrame(train_meta['regions'])
train_reg.columns = ['region_id', 'region_name']
train_df = train_df.merge(train_cat, on='category_id', how='outer')
train_df = train_df.merge(train_img, on='image_id', how='outer')
train_df = train_df.merge(train_reg, on='region_id', how='outer')

In [ ]:
print(train_df.info())
display(train_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030748 entries, 0 to 1030747
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   category_id    1030748 non-null  int64  
 1   id             1030747 non-null  float64
 2   image_id       1030747 non-null  float64
 3   region_id      1030747 non-null  float64
 4   family         1030748 non-null  object 
 5   genus          1030748 non-null  object 
 6   category_name  1030748 non-null  object 
 7   file_name      1030747 non-null  object 
 8   height         1030747 non-null  float64
 9   license        1030747 non-null  float64
 10  width          1030747 non-null  float64
 11  region_name    1030747 non-null  object 
dtypes: float64(6), int64(1), object(5)
memory usage: 102.2+ MB
None


,category_id,id,image_id,region_id,family,genus,category_name,file_name,height,license,width,region_name
0,15672,354106.0,354106.0,1.0,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/354106.jpg,1000.0,1.0,661.0,South America
1,15672,545181.0,545181.0,1.0,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/545181.jpg,1000.0,1.0,661.0,South America
2,15672,449419.0,449419.0,1.0,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/449419.jpg,1000.0,1.0,662.0,South America
3,15672,200223.0,200223.0,1.0,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/200223.jpg,1000.0,1.0,661.0,South America
4,15672,5327.0,5327.0,1.0,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/5327.jpg,1000.0,1.0,661.0,South America
...,...,...,...,...,...,...,...,...,...,...,...,...
1030743,3467,12768.0,12768.0,2.0,Bromeliaceae,Billbergia,Billbergia macrocalyx Hook.,images/034/67/12768.jpg,1000.0,1.0,678.0,US & Canada
1030744,3467,945011.0,945011.0,2.0,Bromeliaceae,Billbergia,Billbergia macrocalyx Hook.,images/034/67/945011.jpg,1000.0,1.0,678.0,US & Canada
1030745,20684,916700.0,916700.0,2.0,Paeoniaceae,Paeonia,Paeonia brownii Douglas ex Hook.,images/206/84/916700.jpg,1000.0,1.0,681.0,US & Canada
1030746,20684,577648.0,577648.0,2.0,Paeoniaceae,Paeonia,Paeonia brownii Douglas ex Hook.,images/206/84/577648.jpg,1000.0,1.0,676.0,US & Canada


In [ ]:
na = train_df.file_name.isna()
keep = [x for x in range(train_df.shape[0]) if not na[x]]
train_df = train_df.iloc[keep]

In [ ]:
dtypes = ['int32', 'int32', 'int32', 'int32', 'object', 'object', 'object', 'object', 'int32', 'int32', 'int32', 'object']
for n, col in enumerate(train_df.columns):
    train_df[col] = train_df[col].astype(dtypes[n])
print(train_df.info())
display(train_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030747 entries, 0 to 1030746
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   category_id    1030747 non-null  int32 
 1   id             1030747 non-null  int32 
 2   image_id       1030747 non-null  int32 
 3   region_id      1030747 non-null  int32 
 4   family         1030747 non-null  object
 5   genus          1030747 non-null  object
 6   category_name  1030747 non-null  object
 7   file_name      1030747 non-null  object
 8   height         1030747 non-null  int32 
 9   license        1030747 non-null  int32 
 10  width          1030747 non-null  int32 
 11  region_name    1030747 non-null  object
dtypes: int32(7), object(5)
memory usage: 74.7+ MB
None


,category_id,id,image_id,region_id,family,genus,category_name,file_name,height,license,width,region_name
0,15672,354106,354106,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/354106.jpg,1000,1,661,South America
1,15672,545181,545181,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/545181.jpg,1000,1,661,South America
2,15672,449419,449419,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/449419.jpg,1000,1,662,South America
3,15672,200223,200223,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/200223.jpg,1000,1,661,South America
4,15672,5327,5327,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg,images/156/72/5327.jpg,1000,1,661,South America
...,...,...,...,...,...,...,...,...,...,...,...,...
1030742,2765,602937,602937,2,Pteridaceae,Astrolepis,Astrolepis windhamii D.M.Benham,images/027/65/602937.jpg,1000,1,678,US & Canada
1030743,3467,12768,12768,2,Bromeliaceae,Billbergia,Billbergia macrocalyx Hook.,images/034/67/12768.jpg,1000,1,678,US & Canada
1030744,3467,945011,945011,2,Bromeliaceae,Billbergia,Billbergia macrocalyx Hook.,images/034/67/945011.jpg,1000,1,678,US & Canada
1030745,20684,916700,916700,2,Paeoniaceae,Paeonia,Paeonia brownii Douglas ex Hook.,images/206/84/916700.jpg,1000,1,681,US & Canada


In [ ]:
test_df = pd.DataFrame(test_meta['images'])
test_df.columns = ['file_name', 'height', 'image_id', 'license', 'width']
print(test_df.info())
display(test_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138292 entries, 0 to 138291
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   file_name  138292 non-null  object
 1   height     138292 non-null  int64 
 2   image_id   138292 non-null  int64 
 3   license    138292 non-null  int64 
 4   width      138292 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 5.3+ MB
None


,file_name,height,image_id,license,width
0,images/104/104891.jpg,1000,104891,1,661
1,images/018/18029.jpg,1000,18029,1,661
2,images/035/35151.jpg,1000,35151,1,661
3,images/124/124144.jpg,1000,124144,1,682
4,images/024/24649.jpg,1000,24649,1,682
...,...,...,...,...,...
138287,images/032/32738.jpg,1000,32738,1,661
138288,images/016/16804.jpg,1000,16804,1,682
138289,images/113/113662.jpg,1000,113662,1,682
138290,images/086/86100.jpg,1000,86100,1,682


In [ ]:
train_df.to_csv('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/full_train_data.csv', index=False)
test_df.to_csv('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/full_test_data.csv', index=False)

# 새 섹션

In [ ]:
train_df = pd.read_csv('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/full_train_data.csv')
test_df = pd.read_csv('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/full_test_data.csv')

In [ ]:
print(train_df.category_id.value_counts())

23718    1765
163      1195
5340     1090
23713    1077
8437     1024
         ... 
29409       2
14021       2
23142       1
24021       1
22922       1
Name: category_id, Length: 32093, dtype: int64


In [ ]:
for col in train_df.columns:
    print("{0:10s} \t {1:10d}".format(col, len(train_df[col].unique())))

category_id 	      32093
id         	    1030747
image_id   	    1030747
region_id  	          4
family     	        310
genus      	       3678
category_name 	      32093
file_name  	    1030747
height     	         32
license    	          1
width      	        181
region_name 	          4


In [ ]:
family = train_df[['family', 'genus', 'category_name']].groupby(['family', 'genus']).count()
display(family.describe())

,category_name
count,3680.000000
mean,280.094293
std,763.666084
min,2.000000
25%,12.000000
50%,54.000000
75%,198.000000
max,14490.000000


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.callbacks import ModelCheckpoint
in_out_size = (120*120) + 3
img_shape = 120
name_of_model = "model1"
def xavier(shape, dtype=None):
    return np.random.rand(*shape)*np.sqrt(1/in_out_size)

def fg_model(shape, lr=0.001):
    i = Input(shape)
    x = Dense(128)
    for j in range(5):
        x = Conv2D(128, (32, 32), activation='relu', kernel_initializer=xavier, padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPool2D((2, 2))(x)
        x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', kernel_initializer=xavier)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    o3 = Dense(2591, activation='softmax', name='category_id', kernel_initializer=xavier)(x)

    x = Model(inputs=i, outputs=o3)

    opt = Adam(lr=lr, amsgrad=True)
    x.compile(optimizer=opt, loss=['sparse_categorical_crossentropy'],
                  metrics=['accuracy'])
    return x

model = fg_model((img_shape, img_shape, 3))
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 120, 120, 32)      896       
_________________________________________________________________
batch_normalization_51 (Batc (None, 120, 120, 32)      128       
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 120, 120, 32)      9248      
_________________________________________________________________
batch_normalization_52 (Batc (None, 120, 120, 32)      128       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 60, 60, 32)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 60, 60, 32)        0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
import numpy as np
group_3 = np.array([1,1, 1, 2])
group_4 = np.array([3,3,3,1])

In [ ]:
def gini(x):
    return 1 - ((x / x.sum())**2).sum()
def entropy(x):
    return (-x*np.log2(x)).sum()
gini_3 = gini(group_3)
gini_4 = gini(group_4)
print(gini_3)
print(gini_4)
print(entropy(group_3))
print(entropy(group_4))

0.72
0.72
-2.0
-14.264662506490405


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(featurewise_center=False,
                                     featurewise_std_normalization=False,
                                     rotation_range=180,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     zoom_range=0.2)

In [ ]:
m = train_df[['file_name', 'family', 'genus', 'category_id']]
fam = m.family.unique().tolist()
m.family = m.family.map(lambda x: fam.index(x))
gen = m.genus.unique().tolist()
m.genus = m.genus.map(lambda x: gen.index(x))
display(m)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,file_name,family,genus,category_id
0,images/156/72/354106.jpg,0,0,15672
1,images/156/72/545181.jpg,0,0,15672
2,images/156/72/449419.jpg,0,0,15672
3,images/156/72/200223.jpg,0,0,15672
4,images/156/72/5327.jpg,0,0,15672
...,...,...,...,...
1030742,images/027/65/602937.jpg,25,1554,2765
1030743,images/034/67/12768.jpg,32,798,3467
1030744,images/034/67/945011.jpg,32,798,3467
1030745,images/206/84/916700.jpg,309,3677,20684


In [ ]:
train, verif = tts(m, test_size=0.2, shuffle=True)#,random_state=17)
print(len(train))
print(len(verif))
print(int(len(train)/10)*(c-1))

NameError: ignored

In [ ]:
c=1 #1~10
train = train[int(len(train)/10)*(c-1):int(len(train)/10)*c]
verif = verif[int(len(verif)/10)*(c-1):int(len(verif)/10)*c]
print(len(train))
shape = (img_shape, img_shape, 3)
epochs = 20
batch_size = 40 #640

DIR_TRAIN='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/'
TRAIN_DIR_TRAIN='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/'
TEST_DIR_TRAIN='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/test/'
print("TYPE",type(train['category_id']))


#train['category_id'] = train['category_id'].astype(str)
#verif['category_id'] = verif['category_id'].astype(str)

print("after train \n", train.dtypes)
print("\nfile name = ", type(train['file_name']))
if c == 1:
    model = fg_model(shape, 0.007)
else:
    model = fg_model(shape, 0.007)
    model.load_weights('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model_id.h5')


print("train",train['category_id'].nunique())
print("verif",verif['category_id'].nunique())


for layers in model.layers:
    if layers.name == 'category_id':
        layers.trainable = True
model_checkpoint_callback = ModelCheckpoint(
    filepath='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model_id.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
history=model.fit_generator(train_datagen.flow_from_dataframe(dataframe=train,
                                                              directory=TRAIN_DIR_TRAIN,
                                                              x_col="file_name",
                                                              y_col="category_id",
                                                              target_size=(img_shape, img_shape),
                                                              batch_size=batch_size,
                                                              class_mode='raw'),
                            validation_data=train_datagen.flow_from_dataframe(
                            dataframe=verif,
                            directory=TRAIN_DIR_TRAIN,
                            x_col="file_name",
                            y_col="category_id",
                            target_size=(img_shape, img_shape),
                            batch_size=batch_size,
                            class_mode='raw'),
                            epochs=epochs,
                            steps_per_epoch=len(train)//batch_size,
                            validation_steps=len(verif)//batch_size,
                            verbose=1)
model.save('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model_id.h5')
model.save('fg_model_id.h5')

82459
TYPE <class 'pandas.core.series.Series'>
after train 
 file_name      object
family          int64
genus           int64
category_id     int64
dtype: object

file name =  <class 'pandas.core.series.Series'>


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Nombre de catégories entrainées dans train 18353
Nombre de catégories entrainées dans verif 9426
Found 82459 validated image filenames.
Found 20615 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1892/2061 [==========================>...] - ETA: 31:14 - loss: 9.7722 - accuracy: 0.0010

In [ ]:

import matplotlib.pyplot as plt
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'test_acc'], loc='upper left')
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'test_loss'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.callbacks import ModelCheckpoint
in_out_size = (120*120) + 3 #We will resize the image to 120*120 and we have 3 outputs
def xavier(shape, dtype=None):
    return np.random.rand(*shape)*np.sqrt(1/in_out_size)

def fg_model(shape, lr=0.001):
    '''Family-Genus model receives an image and outputs two integers indicating both the family and genus index.'''
    i = Input(shape)
    
    x = Conv2D(3, (3, 3), activation='relu', padding='same', kernel_initializer=xavier)(i)
    x = Conv2D(3, (5, 5), activation='relu', padding='same', kernel_initializer=xavier)(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(3,3))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Conv2D(16, (5, 5), activation='relu', padding='same', kernel_initializer=xavier)(x)
    #x = Conv2D(16, (5, 5), activation='relu', padding='same', kernel_initializer=xavier)(x)
    x = MaxPool2D(pool_size=(5, 5), strides=(5,5))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    
    o1 = Dense(310, activation='softmax', name='family', kernel_initializer=xavier)(x)
    
    o2 = concatenate([o1, x])
    o2 = Dense(3678, activation='softmax', name='genus', kernel_initializer=xavier)(o2)
    
    o3 = concatenate([o1, o2, x])
    o3 = Dense(32094, activation='softmax', name='category_id', kernel_initializer=xavier)(o3)
    
    x = Model(inputs=i, outputs=[o1, o2, o3])
    
    opt = Adam(lr=lr, amsgrad=True)
    x.compile(optimizer=opt, loss=['sparse_categorical_crossentropy', 
                                   'sparse_categorical_crossentropy', 
                                   'sparse_categorical_crossentropy'],
                 metrics=['accuracy'])
    return x

model = fg_model((120, 120, 3))
model.summary()
plot_model(model, to_file='full_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
train, verif = tts(m, test_size=0.2, shuffle=True, random_state=17)
train = train[:40000]
verif = verif[:10000]
shape = (120, 120, 3)
epochs = 10
batch_size = 32

model = fg_model(shape, 0.007)

#Disable the last two output layers for training the Family
for layers in model.layers:
    if layers.name == 'genus' or layers.name=='category_id':
        layers.trainable = False

#Train Family for 2 epochs
model_checkpoint_callback = ModelCheckpoint(
    filepath='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
model.fit_generator(train_datagen.flow_from_dataframe(dataframe=train,
                                                      directory='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/',
                                                      x_col="file_name",
                                                      y_col=["family", "genus", "category_id"],
                                                      target_size=(120, 120),
                                                      batch_size=batch_size,
                                                      class_mode='multi_output'),
                    validation_data=train_datagen.flow_from_dataframe(
                        dataframe=verif,
                        directory='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/',
                        x_col="file_name",
                        y_col=["family", "genus", "category_id"],
                        target_size=(120, 120),
                        batch_size=batch_size,
                        class_mode='multi_output'),
                    epochs=epochs,
                    steps_per_epoch=len(train)//batch_size,
                    validation_steps=len(verif)//batch_size,
                    verbose=1,
                    workers = 8,
                    # callbacks=[model_checkpoint_callback]
                    )
model.save('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5')
model.save('fg_model.h5')



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Found 40000 validated image filenames.
Found 10000 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1250/1250 [==============================] - 3563s 3s/step - loss: 21.5697 - family_loss: 4.5053 - genus_loss: 7.0466 - category_id_loss: 10.0178 - family_accuracy: 0.1010 - genus_accuracy: 0.0181 - category_id_accuracy: 0.0014 - val_loss: 21.3124 - val_family_loss: 4.4298 - val_genus_loss: 6.9140 - val_category_id_loss: 9.9686 - val_family_accuracy: 0.1002 - val_genus_accuracy: 0.0087 - val_category_id_accuracy: 0.0017
Epoch 2/10
1250/1250 [==============================] - 634s 504ms/step - loss: 20.1352 - family_loss: 4.2338 - genus_loss: 6.6298 - category_id_loss: 9.2716 - family_accuracy: 0.1110 - genus_accuracy: 0.0232 - category_id_accuracy: 0.0022 - val_loss: 20.9285 - val_family_loss: 4.2484 - val_genus_loss: 6.7072 - val_category_id_loss: 9.9729 - val_family_accuracy: 0.1021 - val_genus_accuracy: 0.0193 - val_category_id_accuracy: 0.0015
Epoch 3/10
1250/1250 [==============================] - 608s 485ms/step - loss: 19.7095 - family_loss: 4.1592 - genus_loss: 6.492

# 새 섹션

In [ ]:
model.save('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5')
model.save('fg_model.h5')

SyntaxError: ignored

# 새 섹션

In [ ]:
from tensorflow import keras
import tensorflow as tf
# model =  tf.saved_model.load('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5',custom_objects=None, compile=True, options=None)
# model =  tf.keras.models.load_model('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5',custom_objects=None, compile=True, options=None)
shape = (120, 120, 3)
epochs = 10
batch_size = 32

# model =  tf.keras.models.load_model('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#Reshuffle the inputs
train, verif = tts(m, test_size=0.2, shuffle=True, random_state=17)
train = train[:40000]
verif = verif[:10000]

#Make the Genus layer Trainable
for layers in model.layers:
    if layers.name == 'genus':
        layers.trainable = True

model_checkpoint_callback1 = ModelCheckpoint(
    filepath='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)



#Train Family and Genus for 2 epochs
model.fit_generator(train_datagen.flow_from_dataframe(dataframe=train,
                                                      directory='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/',
                                                      x_col="file_name",
                                                      y_col=["family", "genus", "category_id"],
                                                      target_size=(120, 120),
                                                      batch_size=batch_size,
                                                      class_mode='multi_output'),
                    validation_data=train_datagen.flow_from_dataframe(
                        dataframe=verif,
                        directory='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train/',
                        x_col="file_name",
                        y_col=["family", "genus", "category_id"],
                        target_size=(120, 120),
                        batch_size=batch_size,
                        class_mode='multi_output'),
                    epochs=epochs,
                    steps_per_epoch=len(train)//batch_size,
                    validation_steps=len(verif)//batch_size,
                    verbose=1,
                    workers = 8,
                    callbacks=[model_checkpoint_callback1]
                    )
model.save('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5')
model.save('fg_model1.h5')


Found 40000 validated image filenames.
Found 10000 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1250/1250 [==============================] - 1841s 1s/step - loss: 21.6289 - family_loss: 4.5390 - genus_loss: 7.0891 - category_id_loss: 10.0008 - family_accuracy: 0.0996 - genus_accuracy: 0.0148 - category_id_accuracy: 0.0012 - val_loss: 21.0930 - val_family_loss: 4.3875 - val_genus_loss: 6.8843 - val_category_id_loss: 9.8212 - val_family_accuracy: 0.1147 - val_genus_accuracy: 0.0149 - val_category_id_accuracy: 2.0032e-04
Epoch 2/10
1250/1250 [==============================] - 649s 517ms/step - loss: 20.2448 - family_loss: 4.2339 - genus_loss: 6.6716 - category_id_loss: 9.3394 - family_accuracy: 0.1276 - genus_accuracy: 0.0239 - category_id_accuracy: 0.0018 - val_loss: 20.8651 - val_family_loss: 4.2363 - val_genus_loss: 6.7164 - val_category_id_loss: 9.9123 - val_family_accuracy: 0.1327 - val_genus_accuracy: 0.0264 - val_category_id_accuracy: 0.0027
Epoch 3/10
1250/1250 [==============================] - 643s 512ms/step - loss: 19.8315 - family_loss: 4.1540 - genus_loss: 6

In [ ]:
from tensorflow import keras
import tensorflow as tf
# model =  tf.saved_model.load('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5')
# model =  tf.keras.models.load_model('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5',custom_objects=None, compile=True, options=None)
shape = (120, 120, 3)
epochs = 10
batch_size = 32
model = fg_model(shape, 0.007)
model.load_weights('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5')
# model =  tf.keras.models.load_model('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#Reshuffle the inputs
train, verif = tts(m, test_size=0.2, shuffle=True, random_state=17)
train = train[:40000]
verif = verif[:10000]
model_checkpoint_callback2 = ModelCheckpoint(
    filepath='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model2.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
#Make the category_id layer Trainable
for layers in model.layers:
    if layers.name == 'category_id':
        layers.trainable = True
        
#Train them all for 2 epochs
model.fit_generator(train_datagen.flow_from_dataframe(dataframe=train,
                                                      directory='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train',
                                                      x_col="file_name",
                                                      y_col=["family", "genus", "category_id"],
                                                      target_size=(120, 120),
                                                      batch_size=batch_size,
                                                      class_mode='multi_output'),
                    validation_data=train_datagen.flow_from_dataframe(
                        dataframe=verif,
                        directory='/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/train',
                        x_col="file_name",
                        y_col=["family", "genus", "category_id"],
                        target_size=(120, 120),
                        batch_size=batch_size,
                        class_mode='multi_output'),
                    epochs=epochs,
                    steps_per_epoch=len(train)//batch_size,
                    validation_steps=len(verif)//batch_size,
                    verbose=1,
                    workers = 8,
                    callbacks=[model_checkpoint_callback2]
                    )
model.save('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model2.h5')
model.save('fg_model2.h5')

Found 40000 validated image filenames.
Found 10000 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1250/1250 [==============================] - 3199s 3s/step - loss: 19.6661 - family_loss: 4.0839 - genus_loss: 6.4202 - category_id_loss: 9.1620 - family_accuracy: 0.1404 - genus_accuracy: 0.0350 - category_id_accuracy: 0.0039 - val_loss: 26.0636 - val_family_loss: 6.2895 - val_genus_loss: 8.4909 - val_category_id_loss: 11.2831 - val_family_accuracy: 0.0722 - val_genus_accuracy: 0.0145 - val_category_id_accuracy: 0.0015
Epoch 2/10
1250/1250 [==============================] - 258s 206ms/step - loss: 19.1721 - family_loss: 4.0950 - genus_loss: 6.3157 - category_id_loss: 8.7614 - family_accuracy: 0.1398 - genus_accuracy: 0.0351 - category_id_accuracy: 0.0044 - val_loss: 22.9853 - val_family_loss: 4.8423 - val_genus_loss: 7.3522 - val_category_id_loss: 10.7907 - val_family_accuracy: 0.0758 - val_genus_accuracy: 0.0204 - val_category_id_accuracy: 0.0027
Epoch 3/10
1250/1250 [==============================] - 259s 206ms/step - loss: 18.9186 - family_loss: 4.0800 - genus_loss: 6.24

In [ ]:
# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지");
#     document.querySelector("colab-toolbar-button#connect").click()
# }
# setInterval(ClickConnect, 60 * 1000)

In [ ]:
from tensorflow import keras
import tensorflow as tf
# model =  tf.saved_model.load('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5')
# model =  tf.keras.models.load_model('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model.h5',custom_objects=None, compile=True, options=None)
shape = (120, 120, 3)
epochs = 10
batch_size = 32
model = fg_model(shape, 0.007)
model.load_weights('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model2.h5')
# model =  tf.keras.models.load_model('/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/fg_model1.h5')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
batch_size = 32
test_datagen = ImageDataGenerator(featurewise_center=False,
                                  featurewise_std_normalization=False)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df, #Limiting the test to the first 10,000 
        # /gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/test
        directory = '/gdrive/MyDrive/dacon/Herbarium/herbarium-2020-fgvc7/nybg2020/test/',
        x_col = 'file_name',
        target_size=(120, 120),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
print(test_generator)
family, genus, category = model.predict_generator(test_generator, verbose=1)

Found 138292 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


 713/4322 [===>..........................] - ETA: 21:00:04

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_df.image_id
sub['Id'] = sub['Id'].astype('int32')
sub['Predicted'] = np.concatenate([np.argmax(category, axis=1), 23718*np.ones((len(test_df.image_id)-len(category)))], axis=0)
sub['Predicted'] = sub['Predicted'].astype('int32')
display(sub)
sub.to_csv('category_submission.csv', index=False)

In [ ]:
sub['Predicted'] = np.concatenate([np.argmax(genus, axis=1), np.zeros((len(test_df.image_id)-len(genus)))], axis=0)
sub['Predicted'] = sub['Predicted'].astype('int32')
display(sub)
sub.to_csv('genus_submission.csv', index=False)